```{contents}
```
## **Prompt Routing in LangGraph**

**Prompt Routing** in LangGraph is a **control mechanism** that dynamically selects **which prompt, model, or tool pipeline** should process an input based on the **current state, task type, or intermediate results**.
It enables **adaptive, multi-skill, multi-model, and multi-agent systems** within a single workflow.

---

### **1. Motivation**

A single static prompt cannot handle all tasks efficiently.
Different tasks require:

| Task Type          | Best Prompt Strategy           |
| ------------------ | ------------------------------ |
| Question answering | Concise factual prompt         |
| Planning           | Step-by-step structured prompt |
| Code generation    | Strict format & constraints    |
| Tool use           | Action-oriented prompt         |
| Safety-critical    | Conservative & verified prompt |

Prompt routing selects the **right prompt at the right time**.

---

### **2. Conceptual Model**

```
User Input → Classify Intent → Select Prompt → Execute → Update State
```

Routing decisions are **state-driven** and **data-dependent**.

---

### **3. Where Prompt Routing Lives in LangGraph**

Prompt routing is implemented using:

* **Router nodes**
* **Conditional edges**
* **State inspection**
* **Dynamic prompt construction**

---

### **4. State Schema Example**

```python
class State(TypedDict):
    input: str
    task_type: str
    result: str
```

---

### **5. Intent Classification Node**

```python
def classify_intent(state: State):
    text = state["input"].lower()
    if "code" in text:
        task = "coding"
    elif "plan" in text:
        task = "planning"
    else:
        task = "qa"
    return {"task_type": task}
```

---

### **6. Prompt Library**

```python
PROMPTS = {
    "qa": "Answer the question concisely:\n{input}",
    "coding": "Write correct Python code:\n{input}",
    "planning": "Create a step-by-step plan:\n{input}"
}
```

---

### **7. Prompt Routing Node**

```python
def apply_prompt(state: State):
    template = PROMPTS[state["task_type"]]
    prompt = template.format(input=state["input"])
    response = llm.invoke(prompt)
    return {"result": response}
```

---

### **8. Routing Logic in the Graph**

```python
builder = StateGraph(State)

builder.add_node("classify", classify_intent)
builder.add_node("execute", apply_prompt)

builder.set_entry_point("classify")
builder.add_edge("classify", "execute")
builder.add_edge("execute", END)
```

---

### **9. Advanced Routing with Conditional Edges**

```python
def route_by_task(state: State):
    return state["task_type"]

builder.add_conditional_edges("classify", route_by_task, {
    "qa": "execute",
    "coding": "execute",
    "planning": "execute"
})
```

---

### **10. Production Variants**

| Variant             | Purpose                         |
| ------------------- | ------------------------------- |
| Rule-based routing  | Simple classification           |
| LLM-based router    | Complex intent detection        |
| Multi-model routing | Cost & performance optimization |
| Safety routing      | High-risk content isolation     |
| Skill routing       | Specialized pipelines           |
| Agent routing       | Multi-agent delegation          |

---

### **11. Real-World Use Cases**

| System                | Benefit                     |
| --------------------- | --------------------------- |
| Enterprise assistants | Task-aware responses        |
| Autonomous agents     | Skill selection             |
| Customer support bots | Intent-based workflows      |
| Research assistants   | Tool specialization         |
| Cost-optimized AI     | Cheap models for easy tasks |

---

### **12. Key Advantages**

| Without Routing    | With Routing               |
| ------------------ | -------------------------- |
| Single weak prompt | Specialized expert prompts |
| High cost          | Cost-optimized execution   |
| Lower accuracy     | Higher task precision      |
| Rigid behavior     | Adaptive intelligence      |

---

### **13. Mental Model**

Prompt routing turns LangGraph into a **mixture-of-experts system**:

> **Input → Expert Selection → Expert Prompt → Execution**

This is a core mechanism behind **scalable, reliable, and intelligent AI systems**.

### Demonstration

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI

# -------------------------------
# 1. Define State
# -------------------------------

class State(TypedDict):
    input: str
    task_type: str
    result: str

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# -------------------------------
# 2. Nodes
# -------------------------------

def classify_intent(state: State):
    text = state["input"].lower()
    if "code" in text:
        task = "coding"
    elif "plan" in text:
        task = "planning"
    else:
        task = "qa"
    return {"task_type": task}

PROMPTS = {
    "qa": "Answer briefly:\n{input}",
    "coding": "Write correct Python code:\n{input}",
    "planning": "Create a step-by-step plan:\n{input}"
}

def apply_prompt(state: State):
    prompt = PROMPTS[state["task_type"]].format(input=state["input"])
    response = llm.invoke(prompt).content
    return {"result": response}

# -------------------------------
# 3. Build Graph
# -------------------------------

builder = StateGraph(State)

builder.add_node("classify", classify_intent)
builder.add_node("execute", apply_prompt)

builder.set_entry_point("classify")
builder.add_edge("classify", "execute")
builder.add_edge("execute", END)

graph = builder.compile()

# -------------------------------
# 4. Test Runs
# -------------------------------

tests = [
    "Explain what transformers are",
    "Write code to reverse a list",
    "Create a project plan for a mobile app"
]

for t in tests:
    print("\nInput:", t)
    output = graph.invoke({"input": t})
    print("Detected task:", output["task_type"])
    print("Result:\n", output["result"])



Input: Explain what transformers are
Detected task: qa
Result:
 Transformers are a type of neural network architecture designed for processing sequential data, particularly in natural language processing tasks. They use a mechanism called self-attention to weigh the importance of different words in a sequence, allowing them to capture long-range dependencies and relationships. Transformers consist of an encoder-decoder structure, where the encoder processes the input data and the decoder generates the output. They have become the foundation for many state-of-the-art models, such as BERT and GPT.

Input: Write code to reverse a list
Detected task: coding
Result:
 You can reverse a list in Python using several methods. Here are a few common approaches:

### Method 1: Using the `reverse()` method
This method reverses the list in place.

```python
my_list = [1, 2, 3, 4, 5]
my_list.reverse()
print(my_list)  # Output: [5, 4, 3, 2, 1]
```

### Method 2: Using slicing
This method creates a ne